                                        PREDICT THE ODDS OF WINNING OF THE GAME

1. Crawl data from the page
    Link: https://www.leagueofgraphs.com/

Step 0: Import the necessary libraries

In [97]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json, codecs

Step 1: Open the page
    - Use Request 
    - Use headers attribute: 'User-Agent'='Mozilla/5.0' 

In [12]:
url = "https://www.leagueofgraphs.com"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(req).read()

Step 2: Read and convert page to html format
    - Use BeautifulSoup 

In [14]:
soup = BeautifulSoup(page,'html.parser')
soup


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,500,700" media="none" onload="this.media='all'" rel="stylesheet" type="text/css"/> <link href="//lolg-cdn.porofessor.gg/style.css?v=1655828821" rel="stylesheet" type="text/css">
<link href="//lolg-cdn.porofessor.gg/style.sprite.css?v=de35750e941d9e3f34243a3abd186033" rel="stylesheet" type="text/css"/>
<script src="//lolg-cdn.porofessor.gg/jquery.js?v=3"></script>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="We track the millions of LoL games played every day to gather champion stats, matchups, builds &amp; summoner rankings, as well as champion stats, popularity, winrate, teams rankings, best items and spells." name="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatib

Step 3: Find the rankings link
    - Section "rankings" in "data-link-section" of tag a
    - "data-link-section" in "sidebar-navigation accordion-navigation" class

In [61]:
main_menu = soup.find_all(class_="sidebar-navigation accordion-navigation")
rankings_link = ''
url_size = 0
for _ in main_menu:
    
    a = _.find("a")
    section = a.get("data-link-section")
    print(section)
    if section == 'rankings':
        rankings_link = url+a.get("href")
        url_size = len(rankings_link[:len(url)])
        print(rankings_link)

champions
rankings
https://www.leagueofgraphs.com/rankings/summoners
replays


Step 4: Crawl information of players and their games, include:
    1. Get the link of ranking pages 
    2. Get the link of players in each page (100 players per page) 
    3. Get player's information
        - Ranked
        - Win/Loss games
        - ELO point
    4. Get the link of games played (history) of each player (10 latest games)
    5. Get game's information
        - What team won the game
        - K/D/A
        - Duration
        - Champions
        - Ranks
        - Other (damage, gold earned, ...)

4.1 Get the link of ranking pages

In [98]:
def get_page_links(url, url_size):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = BeautifulSoup(page,'html.parser')
    pagi = soup.find(class_="pagination")
    pagi_items = pagi.find_all("li")
    url = url[:url_size]
    for _ in pagi_items:
        li = _.get("class")
        if type(li) == list:
            if "current-page" in li:
                link = url+_.find("a").get("href")
        if _ == pagi_items[-1]:
            next_link = url+_.find("a").get("href")
    return link, next_link

In [64]:
page1, page2 = get_page_links(rankings_link, url_size)
page1, page2

('https://www.leagueofgraphs.com/rankings/summoners',
 'https://www.leagueofgraphs.com/rankings/summoners/page-2')

4.2 Get the link of players in each page (100 players per page)

In [99]:
def get_player_links(url, url_size):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = BeautifulSoup(page,'html.parser')
    player_table = soup.find(class_="data_table summonerRankingsTable with_sortable_column")
    player_items = player_table.find_all("tr")
    player_links = []
    for _ in player_items:
        a = _.find_all("a")
        for _ in a:
            link = _.get("href")
            if link.find('/rankings') != -1:
                continue
            else:
                player_links.append((url[:url_size]+link))
    return player_links

In [79]:
# In page 1
player_links_page_1 = get_player_links(page1, url_size)
player_links_page_1

['https://www.leagueofgraphs.com/summoner/br/Morttheusz',
 'https://www.leagueofgraphs.com/summoner/eune/I+am+Sobek',
 'https://www.leagueofgraphs.com/summoner/eune/Snoopy',
 'https://www.leagueofgraphs.com/summoner/br/QUINDINHO',
 'https://www.leagueofgraphs.com/summoner/eune/Dealersz+R1',
 'https://www.leagueofgraphs.com/summoner/br/VRX+Disamis',
 'https://www.leagueofgraphs.com/summoner/tr/TwTv+tuokaZlol',
 'https://www.leagueofgraphs.com/summoner/las/1Fresco',
 'https://www.leagueofgraphs.com/summoner/euw/MagiFelix5',
 'https://www.leagueofgraphs.com/summoner/eune/T%C3%BAr%C3%B3st%C3%A9szta',
 'https://www.leagueofgraphs.com/summoner/las/FIX+10',
 'https://www.leagueofgraphs.com/summoner/br/Nick+link',
 'https://www.leagueofgraphs.com/summoner/na/insanity123',
 'https://www.leagueofgraphs.com/summoner/kr/JUGKlNG',
 'https://www.leagueofgraphs.com/summoner/kr/LSB+Duro',
 'https://www.leagueofgraphs.com/summoner/kr/SMB+heixiaohu',
 'https://www.leagueofgraphs.com/summoner/euw/good+in

4.3 Get player's information

In [100]:
def get_info_player(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup_player = BeautifulSoup(page,'html.parser')
    league_tier = soup_player.find(class_="leagueTier").get_text().replace("\n", '').strip()
    league_points = soup_player.find(class_="leaguePoints").get_text().replace("\n", '').strip()
    wins_number = soup_player.find(class_="winsNumber").get_text().replace("\n", '').strip()
    losses_number = soup_player.find(class_="lossesNumber").get_text().replace("\n", '').strip()
    
    player_info = {
        "link": url,
        "rank": league_tier[:league_tier.index(' ')],
        "elo": league_points[:league_points.index(' ')],
        "wins": wins_number,
        "losses": losses_number,
    }
    return player_info, soup_player

In [88]:
# Player 1
player_info_1, soup_player = get_info_player(player_links_page_1[0])
print(player_info_1)
# Player 2
player_info_2, soup_player = get_info_player(player_links_page_1[1])
print(player_info_2)

{'link': 'https://www.leagueofgraphs.com/summoner/br/Morttheusz', 'rank': 'Challenger', 'elo': '1857', 'wins': '386', 'losses': '306'}
{'link': 'https://www.leagueofgraphs.com/summoner/eune/I+am+Sobek', 'rank': 'Challenger', 'elo': '1731', 'wins': '368', 'losses': '300'}


4.4 Get the link of games played (history) of each player (10 latest games)

In [101]:
def get_game_links(url, soup_player, url_size):
    history = soup_player.find(class_="data_table relative recentGamesTable inverted_rows_color")
    history_items = history.find_all(class_="resultCellDark nopadding")
    history_list = []
    for _ in history_items:
        a = _.find("a")
        if a != None:
            history_list.append(url[:url_size]+a.get("href"))
    return history_list

In [90]:
history_list_player_1 = get_game_links(player_links_page_1[0], soup_player, url_size)
history_list_player_1

['https://www.leagueofgraphs.com/match/eune/3157167039#participant3',
 'https://www.leagueofgraphs.com/match/eune/3153413658#participant1',
 'https://www.leagueofgraphs.com/match/eune/3153449948#participant2',
 'https://www.leagueofgraphs.com/match/eune/3153076219#participant2',
 'https://www.leagueofgraphs.com/match/eune/3153020062#participant7',
 'https://www.leagueofgraphs.com/match/eune/3152767172#participant6',
 'https://www.leagueofgraphs.com/match/eune/3152675286#participant1',
 'https://www.leagueofgraphs.com/match/eune/3152426051#participant6',
 'https://www.leagueofgraphs.com/match/eune/3152442776#participant9',
 'https://www.leagueofgraphs.com/match/eune/3152264593#participant7']

4.5 Get game's information

In [103]:
def get_info_game(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = BeautifulSoup(page,'html.parser')

    # get table result
    table = soup.find(id="match_datatable")
    champs = table.find_all("img")
    champs_data = []
    for _ in champs:
        name = _.get('alt')
        champs_data.append(name)
    data = [i for i in table.get_text().replace(" ", '').split('\n') if i != '']
    data.remove("Combat")
    data.remove("Damagedealt")
    data.remove("Damagetakenandhealed")
    data.remove("Vision")
    data.remove("Income")
    data.remove("Miscellaneous")

    # get result
    th = soup.find("th")
    result_kda = th.get_text().replace("\n", ' ').strip()

    # get rank
    rows = soup.find_all(class_="playerRow")
    tag_i = []
    for _ in rows:
        tag_i.extend(_.find_all("i"))
    ranks = []
    for _ in tag_i:
        ranks.append(_.get_text().replace("\n", '').strip())

    # final result
    game_info = {
        "result_blue_team": result_kda[:result_kda.index(' ')],
        "kda_blue_team": result_kda[result_kda.index(' ')+1:],
        "game_duration": soup.find(class_="gameDuration").get_text().replace("\n", '').strip(),

        "champs": champs_data,
        "ranks": ranks,

        "largest_killing_spree": data[1:11],
        "largest_multi_kill": data[12:22],
        "crowd_control_score": data[23:33],

        "total_damage_to_champions": data[36:46],
        "physical_damage_to_champions": data[47:57],
        "magic_damage_to_champions": data[58:68],
        "true_damage_to_champions": data[69:79],
        "total_damage_dealt": data[80:90],
        "physical_damage_dealt": data[91:101],
        "magic_damage_dealt": data[102:112],
        "true_damage_dealt": data[113:123],
        "total_damage_to_turrets": data[124:134],
        "total_damage_to_objectives": data[135:145],

        "damage_healed": data[146:156],
        "damage_taken": data[157:167],
        "physical_damage_taken": data[168:178],
        "true_damage_taken": data[179:189],
        "self_mitigated_damage": data[190:200],

        "vision_score": data[201:211],
        "wards": data[212:222],
        "wards_killed": data[223:233],
        "control_wards_purchased": data[234:244],

        "gold_earned": data[245:255],
        "gold_spent": data[256:266],
        "minions_killed": data[267:277],
        "neutral_minions_killed": data[278:288],

        "towers_destroyed": data[289:299],
        "inhibitors_destroyed": data[300:310],
    }

    return game_info

In [84]:
# Game
game_1_info = get_info_game(history_list_player_1[0])
game_1_info

{'result_blue_team': 'Victory',
 'kda_blue_team': '33 / 24 / 63',
 'game_duration': '(28:15)',
 'champs': ['Pantheon',
  'Volibear',
  'Cassiopeia',
  'Miss Fortune',
  'Leona',
  'Fiora',
  'Nocturne',
  'Katarina',
  'Varus',
  'Senna'],
 'ranks': ['Diamond I',
  'Master',
  'Diamond I',
  'Master',
  'Challenger',
  'Master',
  'Master',
  'Master',
  'Diamond I',
  'Master'],
 'largest_killing_spree': ['3', '3', '5', '5', '0', '2', '2', '4', '5', '0'],
 'largest_multi_kill': ['1', '2', '2', '1', '1', '1', '1', '2', '2', '1'],
 'crowd_control_score': ['23',
  '24',
  '17',
  '20',
  '39',
  '7',
  '167',
  '4',
  '28',
  '36'],
 'total_damage_to_champions': ['27.7k',
  '16k',
  '20.3k',
  '30.9k',
  '8.4k',
  '14.8k',
  '12.1k',
  '18.9k',
  '14.4k',
  '12.8k'],
 'physical_damage_to_champions': ['25.4k',
  '9.9k',
  '523',
  '23.9k',
  '2.9k',
  '9.1k',
  '10.5k',
  '5.1k',
  '9.7k',
  '11.9k'],
 'magic_damage_to_champions': ['1.2k',
  '5k',
  '19.8k',
  '5.2k',
  '4.4k',
  '228',
 

Step 5: Dataset
    - Save as Json file
    - 500 players 
    - 3000 games

In [112]:
def get_all_players(link, url_size, num_players):
    player_links = []
    for i in range(int(num_players/100)):
        current_link, next_link = get_page_links(link, url_size)
        links = get_player_links(current_link, url_size)
        for _ in links:
            player_links.append(_)
        link = next_link
    return player_links

In [122]:
def crawl(player_links, url_size, num_games):
    players_info = []
    games_info = []
    count = 0
    try:
        for _ in player_links:
            print('Player:',_)
            player_info, soup_player = get_info_player(_)
            players_info.append(player_info)
            if count < int(num_games/10):
                history_list = get_game_links(_, soup_player, url_size)
                for _ in history_list:
                    print('Game:',_)
                    game_info = get_info_game(_)
                    games_info.append(game_info)
            count += 1
    except:
        return players_info, games_info     
    return players_info, games_info

In [123]:
url = "https://www.leagueofgraphs.com"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(req).read()
soup = BeautifulSoup(page,'html.parser')
main_menu = soup.find_all(class_="sidebar-navigation accordion-navigation")
players_data = {}
games_data = {}
num_players = 300
num_games = 2000
for _ in main_menu:
    
    a = _.find("a")
    section = a.get("data-link-section")
    if section == 'rankings':
        link = url+a.get("href")
        url_size = len(link[:len(url)])
        player_links = get_all_players(link, url_size, num_players)
        players_info, games_info = crawl(player_links, url_size, num_games)
        games_data.update({'games': games_info})
        players_data.update({'players': players_info})

with open('players_data.json', 'wb') as f:
    json.dump(players_data, codecs.getwriter('utf-8')(f), ensure_ascii=False, indent=4)
with open('games_data.json', 'wb') as f:
    json.dump(games_data, codecs.getwriter('utf-8')(f), ensure_ascii=False, indent=4)

Player: https://www.leagueofgraphs.com/summoner/br/Morttheusz
Game: https://www.leagueofgraphs.com/match/br/2539339513#participant8
Game: https://www.leagueofgraphs.com/match/br/2539336317#participant4
Game: https://www.leagueofgraphs.com/match/br/2539340774#participant9
Game: https://www.leagueofgraphs.com/match/br/2539251735#participant9
Game: https://www.leagueofgraphs.com/match/br/2538131109#participant9
Game: https://www.leagueofgraphs.com/match/br/2535772932#participant7
Game: https://www.leagueofgraphs.com/match/br/2535702231#participant3
Game: https://www.leagueofgraphs.com/match/br/2534917618#participant4
Game: https://www.leagueofgraphs.com/match/br/2534852451#participant4
Game: https://www.leagueofgraphs.com/match/br/2534361653#participant8
Player: https://www.leagueofgraphs.com/summoner/eune/I+am+Sobek
Game: https://www.leagueofgraphs.com/match/eune/3157167039#participant3
Game: https://www.leagueofgraphs.com/match/eune/3153413658#participant1
Game: https://www.leagueofgrap